In [ ]:

#IMPORTING NECESSARY LIBRARIES
import cv2
import numpy as np

In [ ]:
# LOADING THE YOLO MODEL
net = cv2.dnn.readNet("yolov3.weights", "yolov3.cfg")

In [ ]:
# LOADING THE FILE WITH CLASSES
with open("coco128.yaml", "r") as f:

    classes = f.read().strip().split("\n")


In [ ]:
# OPENING THE VIDEO FILE AND READING THE FILE FRAME BY FRAME
video_capture = cv2.VideoCapture('trafficjam.mp4')

while True:
    ret, frame = video_capture.read()

    if not ret:
        break


In [ ]:
 # PERFORMING OBJECT DETECTION
    blob = cv2.dnn.blobFromImage(frame, 1/255.0, (416, 416), swapRB=True, crop=False) #CONVERTS A FRAME INTO A BLOB FOR COMPATIBILTY WITH DNN
    net.setInput(blob) #PREPROCESSES THESE FRAMES OR BLOBS
    outs = net.forward(net.getUnconnectedOutLayersNames()) #PREPROCESSES THE BLOBS AND STORED THE PROCESSED FRAMES IN THE OUTS VARIABLE

In [ ]:
conf_threshold = 0.3 #CONFIDENCE THRESHOLD (THE CONTOURS BELOW THIS THRESHOLD WILL BE IGNORED)
    nms_threshold = 0.5
    boxes = []
    for out in outs: #ITERATES OVER EACH OUTPUT FRAME
        for detection in out:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > conf_threshold:
                center_x, center_y, width, height = (detection[0:4] * np.array([frame.shape[1], frame.shape[0], frame.shape[1], frame.shape[0]]))
                x, y = int(center_x - width / 2), int(center_y - height / 2)
                boxes.append([x, y, int(width), int(height)])

In [ ]:
  # DRAWING BOUNDING BOXES ON THE VIDEO FRAME
    for box in boxes:
        x, y, w, h = box
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)

In [ ]:
#DEFINING THE CINGESTION DENSITY AND IT'S THRESHOLD TO CLASSIFY THE ROAD IN THE FOOTAGE AS CONGESTED OR NOT CONGESTED
    crowd_density = len(boxes) #DENSITY IS EQUAL TO THE NUMBER OF BOXES PLOTTED OR THE NUMBER PF CARS DETECTED ON THE ROAD
    congestion_threshold = 40  #THRESHOLD SET TO 40 BOUNDING BOXES .I.E. 40 DETECTED CARS ON A ROAD

In [ ]:
#SETTING A CONDITION TO CLASSIFY THE ROAD IN THE VIDEO AS CONGESTED OR NOT CONGESTED
if crowd_density > congestion_threshold:
        congestion_status = "Congested"
        frame = cv2.putText(frame, f"Congestion: {congestion_status}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
else:
        congestion_status = "Not Congested"
        frame = cv2.putText(frame, f"Congestion: {congestion_status}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)


In [ ]:
 #DISPLAYING THE FINAL OUTPUT FRAME
    cv2.imshow("Object Detection", frame)

    if cv2.waitKey(30) & 0xFF == ord('q'):
        break

In [ ]:

#SENDING A BROADCAST MESSAGE USING WHATSAPP TO USERS IF THE ROAD IS CONGESTED USING PYWHATKIT LIBRARY
import pywhatkit
lst=['+91----------','+91----------']
for i in lst:
     if congestion_status == 'Congested':
        pywhatkit.sendwhatmsg_instantly(i,"The road:XYZ is congested, if you are planning to travel through this route, you are advised to take an alternate path to avoid congestion",4,True,4)